In [10]:
from sage.all import *
import networkx as nx
import matplotlib.pyplot as plt
from math import *
from itertools import *

For permutations $\sigma \in \mathfrak{S}_n$ of normal form $NF(\sigma) = (j_1, \dots, j_{n-1})$ satisfying the normal form condition
$$ \forall i,\,j_{i+1} \leq j_i + 1$$
We have, $|[e, \sigma]| = \prod_i (j_i + 1)$

In [ ]:
def length_Bruhat_lower_interval(w,W): 
    """
    W : WeylGroup Sage class
    w : product of simple reflexions in WeylGroup Sage class
    Return the length of the Bruhat-interval [e,w] in W
    Naive algorithm
    """
    return sum(1 for v in W if v.bruhat_le(w))

def NF_2_elem(NF):
    """
    NF = [j1, ... , j_{n-1}]  (0<=j_i<=i)
    return the corresponding elemement in (W,S)
    """
    return  prod([prod([s[Integer(i - j)] for j in range(NF[i])]) for i in range(len(NF))])
    
# The NF satisying j_{i+1} <= j_{i}+1

def generate_suites(n, current=None, i=0): 
    """
    list of all NF satisying j_{i+1} <= j_{i}+1 in Sn+2
    """
    if current is None:
        results = []
        for j0 in range(0, 2):
            results += generate_suites(n, current=[j0], i=1)
        return results
    if i > n:
        return [current]
    j_prev = current[-1]
    max_val = min(i + 1, j_prev + 1)
    suites = []
    for j_next in range(0, max_val + 1):
        suites += generate_suites(n, current + [j_next], i + 1)
    return suites

The (co)exponents of a permutation $NF(\sigma) = (j_1, \dots, j_{n-1})$ satisfying the normal form condition are the $j_i$

In [6]:
R = PolynomialRing(QQ, name='q')
q = R.gen()
W = SymmetricGroup(Integer(5))
s1, s2, s3, s4 = W.simple_reflections()
a = W.one()
b = s1*s2*s1*s3*s2*s1*s4*s3


def length_Bruhat_lower_interval(w,W):
    """
    Return the length of the Bruhat-interval [e,w] in W
    !!!Naive algorithm!!!
    """
    return sum(1 for v in W if v.bruhat_le(w))


def Poincare_polynomial(w):
    """
    return the Poincaré polynomial of w in Coxeter group W
    !!!Naive algorithm!!!
    """
    Dw = []
    for v in W:
        if  v.bruhat_le(w):
            Dw.append(v)
    return sum([q**(v.length()) for v in Dw])

Poincare_polynomial(b) == (1+q)*(1+q+q**2)*(1+q+q**2+q**3)*(1+q+q**2)

True

$w\in\mathfrak{S}_n$, the Schubert variety $X_w$ in $SL(n)/B$ is smooth $\Leftrightarrow$ $w$ avoids $3412$ and $4231$ $\Leftrightarrow$ $P_w(X) = R_w(X)$. In this case,$$|[e, w]| = \prod_i (d_i + 1),$$ where the $d_i$ are the exponents of $w$

In [7]:
def list_index_increasing(L):
    """
    L : List(Integers)
    return [i_1, ... , i_k] such that [L[i_1], ... , L[i_k]] is sorted
    """
    Ls = sorted(L)
    return [L.index(Ls[i])+1 for i in range(len(Ls))]


def avoidance(w,v):
    """
    w in Sl and v in Sk with k<=l given w = [w(1), ..., w(l)], v = [v(1), ..., v(k)]
    Return True if w avoids v, False otherwise
    """
    l = len(v)
    K = [i+1 for i in range(len(w))]
    for C in combinations(K, l):
        wC = [w[i-1] for i in list(C)]
        if list_index_increasing(wC) == v: # same relative order
            return False     
    return True

def number_smooth_permutation(n):
    """
    return the number of smooth Schubert varieties for the Bruhat decomposistion of SLn
    (Weyl group of type A) with 3412 and 4231 avoidance (n>=4)
    """
    K = [i+1 for i in range(n)]
    Sn = list(permutations(K))
    return sum([1 for s in Sn if avoidance(s,[3,4,1,2]) and avoidance(s,[4,2,3,1])])

The permutation satisfying the NF condition are smooth

In [11]:

def simple_reflexion(i, L):
    """
    return the action of the i-th simple reflexion of Sn on L
    (len(L) = n, 0 <= i <= n-1)
    """
    L = L.copy()
    L[i], L[i+1] = L[i+1], L[i]
    return L

def NF_to_list(NF): # /!\ caution to the composition sense /!\ 
    """
    NF(s) = (j1, ..., jn-1), with 0 <= ji <= i 
    return [s(1), ... , s(n)]
    """
    n = len(NF) + 1 ; L = [i + 1 for i in range(n)]
    for i in reversed(range(len(NF))): 
        for j in reversed(range(NF[i])):
            L = simple_reflexion(i-j, L)
    return L
    

for m in range(1,8):
    n = m+2
    print('n =', n)
    G = generate_suites(m) # (satisfying the NF condition in Sm+2)
    LL = [NF_to_list(l) for l in G] ;  count = 0
    for usual_form in LL:
        if not (avoidance(usual_form,[3,4,1,2]) and avoidance(usual_form,[4,2,3,1])): # not smooth
            count += 1
    print("# NF condition and not smooth:", count)       
    print('-'*20)

n = 3
# NF condition and not smooth: 0
--------------------
n = 4
# NF condition and not smooth: 0
--------------------
n = 5
# NF condition and not smooth: 0
--------------------
n = 6
# NF condition and not smooth: 0
--------------------
n = 7
# NF condition and not smooth: 0
--------------------
n = 8
# NF condition and not smooth: 0
--------------------
n = 9
# NF condition and not smooth: 0
--------------------


Normal form of smooth permutations

In [14]:
def generate_all_NF(n):
    """
    generate all sequences (j1, ..., jn-1) with 0 <= ji <= i
    """
    R = [range(i+1) for i in range(1,n)]
    return [list(seq) for seq in product(*R)]

def list_NF_of_smooth(n):
    """
    return the list of all normal forms of smooth permutations
    """
    NFs = generate_all_NF(n) ; smooths = []
    for normal_form in NFs:
        usual_form = NF_to_list(normal_form) 
        if avoidance(usual_form,[3,4,1,2]) and avoidance(usual_form,[4,2,3,1]): # smooth permutation
            smooths.append(normal_form)
    return smooths

def generate_all_NF(n):
    """
    generate all sequences (j1, ..., jn-1) with 0 <= ji <= i
    """
    R = [range(i+1) for i in range(1,n)]
    return [list(seq) for seq in product(*R)]

def list_NF_of_smooth(n):
    """
    return the list of all normal forms of smooth permutations
    """
    NFs = generate_all_NF(n) ; smooths = []
    for normal_form in NFs:
        usual_form = NF_to_list(normal_form) 
        if avoidance(usual_form,[3,4,1,2]) and avoidance(usual_form,[4,2,3,1]): # smooth permutation
            smooths.append(normal_form)
    return smooths

def NF_to_prod_simple_reflections(NF):
    """
    Given a normal form NF of the permuation, 
    return the element in terms of product on simple reflexion 
    in the class sage.categories.coxeter_groups.CoxeterGroups
    """
    n = len(NF) + 1
    W = SymmetricGroup(Integer(n))
    s = W.simple_reflections() 
    
    n = len(NF) + 1 ; perm = W.one()
    for i in range(len(NF)): 
        for j in range(NF[i]):
            perm = perm*s[i-j+1]
    return perm

n = 5 ; W = SymmetricGroup(Integer(n)) 
NF_smooth_perm = list_NF_of_smooth(n)
